In [20]:
import os
os.chdir(r"C:\\Users\\JATIN KUMAR\\Downloads\\python analysis\\Vendor performance analysis\\data\\data")
    

In [ ]:
 import pandas as pd
import sqlite3
import logging
import os
from ingestion_db import ingest_db

logging.basicConfig(
    filename = "logs/get_vendor_summary.log",
    level = logging.DEBUG,
    format = "%(asctime)s - %(levelname)s - %(message)s",
    filemode ="a"
)    
   

In [ ]:
def create_vendor_summary(conn):

    """this function will merge the different tables to get overall vendor summary and adding new columns in the resultant tabale"""
    vendor_sales_summary = pd.read_sql_query("""
      With FreightSummary AS (
      Select VendorNumber ,
      sum(Freight) as FreightCost 
      from vendor_invoice 
      Group by VendorNumber 
    ),
 
    PurchaseSummary AS (
    select 
    p.Brand,
    p.VendorNumber,
    p.VendorName,
    p.PurchasePrice,  
    P.Description,
    pp.Volume,
    pp.Price as ActualPrice,
    sum(p.Quantity) as TotalPurchaseQuantity,
    sum(p.Dollars) as TotalPurchaseDollars
    From purchases p
    join  purchase_prices pp 
    on p.Brand = pp.Brand 
    where p.PurchasePrice > 0
    group by p.VendorNumber,p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Volume, pp.Price
    ), 


   SalesSummary AS (
   select 
   VendorNo,
   Brand,
   sum(SalesQuantity) as TotalSalesQuantity,
   sum(SalesDollars) as TotalSalesDollars,
   sum(SalesPrice) as TotalSalesPrice,
   sum(ExciseTax) as TotalExciseTax
   from sales
   group by VendorNo, Brand 
   )

   SELECT  
   ps.VendorNumber,
   ps.VendorName,
   ps.Brand,
   Ps.Description,
   ps.PurchasePrice,
   ps.ActualPrice,
   ps.Volume,
   ps.TotalPurchaseQuantity,
   ps.TotalPurchaseDollars, 
   ss.TotalSalesQuantity,
   ss.TotalSalesDollars,
   ss.TotalSalesPrice,
   ss.TotalExciseTax,
   fs.FreightCost
   FROM PurchaseSummary as ps
   LEFT JOIN SalesSummary as ss
   on ps.VendorNumber = ss.VendorNo
   AND ps.Brand = ss.Brand
   LEFT JOIN FreightSummary as fs
   on ps.VendorNumber =fs.VendorNumber
   ORDER BY ps.TotalPurchaseDollars DESC""", conn)
   
    return vendor_sales_summary

In [10]:
def clean_data(df):
    '''this function will clean the data'''
    # changing datatype to float
    df['Volume'] = df['Volume'].astype('float')

    # filling missing value with 0
    df.fillna(0, inplace = True)

    # removing spaces from categorical columns
    df['VendorName'] = df['VendorName'].str.strip()
    df['Description'] = df['Description'].str.strip()

    # creating new columns for better analysis
    vendor_sales_summary['GrossProfit'] = vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary['TotalPurchaseDollars']
    vendor_sales_summary['ProfitMargin'] = (vendor_sales_summary['GrossProfit'] / vendor_sales_summary['TotalSalesDollars'])*100
    vendor_sales_summary['StockTurnover'] = vendor_sales_summary['TotalSalesQuantity'] / vendor_sales_summary['TotalPurchaseQuantity']
    vendor_sales_summary['SalesToPurchaseRatio'] = vendor_sales_summary['TotalSalesDollars'] / vendor_sales_summary['TotalPurchaseDollars']
    
    return df


if __name__ == '__main__':
    # creating database connection
    conn = sqlite3.connect("C:\\Users\\JATIN KUMAR\\Downloads\\python analysis\\Vendor performance analysis\\data\\data\\inventory.db")

    logging.info('Creating Vendor Summary Table.....')
    summary_df = create_vendor_summary(conn)
    logging.info(summary_df.head())

    logging.info('Cleaning Data.....')
    clean_df = clean_data(summary_df)
    logging.info(clean_df.head())

    logging.info('Ingesting data.....')
    ingest_db(clean_df, 'vendor_sales_summary', conn)
    logging.info('Completed')

